# Interactive Moment Map Explorer

This notebook demonstrates the use of the `moment-explorer` package for interactive moment map visualization using Plotly.

In [ ]:
# Install the package in development mode (run once)
# !pip install -e ..

In [ ]:
import os
import sys

# Add the src directory to the path (for development)
sys.path.insert(0, os.path.abspath('../src'))

from moment_explorer import create_interactive_explorer

## Configuration: Available Cubes

Define the paths to your FITS cubes and masks.

In [ ]:
base_path = '/Users/jea/AATau/cleaning_data/'

# Available cubes
available_cubes = {
    'N2H+ (2013SG2)': {
        'cube': os.path.join(base_path, 'fits_2013SG2/AATau_N2H+_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2013SG2/AATau_N2H+_clean0_contsub.mask.fits'),
        'name': 'N$_2$H$^+$'
    },
    'HCN (2013SG1)': {
        'cube': os.path.join(base_path, 'fits_2013SG1/AATau_HCN_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2013SG1/AATau_HCN_clean0_contsub.mask.fits'),
        'name': 'HCN'
    },
    'HCO+ (2013SG1)': {
        'cube': os.path.join(base_path, 'fits_2013SG1/AATau_HCO+_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2013SG1/AATau_HCO+_clean0_contsub.mask.fits'),
        'name': 'HCO$^{+}$'
    },
    '13CO (2015)': {
        'cube': os.path.join(base_path, 'fits_2015/AATau_13CO_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2015/AATau_13CO_clean0_contsub.mask.fits'),
        'name': '$^{13}$CO'
    },
    'C18O (2015)': {
        'cube': os.path.join(base_path, 'fits_2015/AATau_C18O_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2015/AATau_C18O_clean0_contsub.mask.fits'),
        'name': 'C$^{18}$O'
    },
    'CN SPW1 (2015)': {
        'cube': os.path.join(base_path, 'fits_2015/AATau_CN_SPW1_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2015/AATau_CN_SPW1_clean0_contsub.mask.fits'),
        'name': 'CN J=7/2-5/2'
    },
    'CN SPW3 (2015)': {
        'cube': os.path.join(base_path, 'fits_2015/AATau_CN_SPW3_clean1_contsub.fits'),
        'mask': os.path.join(base_path, 'fits_2015/AATau_CN_SPW3_clean0_contsub.mask.fits'),
        'name': 'CN J=5/2-3/2'
    }
}

print("Available cubes:")
for key in available_cubes.keys():
    print(f"  - {key}")

## Interactive Explorer

Select a cube from the list above and create an interactive explorer.

### Features:
- **Persistent zoom/pan**: Your view is preserved when updating parameters
- **Debounced updates**: Enable "Auto-apply" for smooth, delayed updates (~200ms)
- **Manual control**: Disable "Auto-apply" and use the Apply button for precise control
- **Multiple moment types**: M0 (integrated intensity), M1 (velocity), M8 (peak), M9 (linewidth)
- **Color scale options**: Viridis, Cividis, Magma, Plasma, RdBu_r, Jet
- **Save to FITS**: Export your current moment map

In [ ]:
# Select a cube
selected_cube = 'N2H+ (2013SG2)'  # Change this to explore different cubes

cube_info = available_cubes[selected_cube]
cube_path = cube_info['cube']
mask_path = cube_info['mask']

# Create the interactive explorer
# Set enable_prefix_sums=True for faster M0/M1 computation (uses more memory)
explorer, ui = create_interactive_explorer(
    cube_path=cube_path,
    mask_path=mask_path,
    enable_prefix_sums=True
)

## Advanced Usage

You can also use the package programmatically without the UI.

In [ ]:
from moment_explorer import MomentMapExplorer
import matplotlib.pyplot as plt

# Create explorer
explorer_adv = MomentMapExplorer()
explorer_adv.load_cube(cube_path, mask_path)

# Generate a moment map
moment_map, compute_time = explorer_adv.generate(
    moment='M0',
    first=30,
    last=70,
    clip_sigma=3.0,
    use_mask=True
)

print(f"Compute time: {compute_time*1000:.1f} ms")

# Plot with matplotlib
fig, ax = plt.subplots(figsize=(8, 7))
im = ax.imshow(moment_map, origin='lower', cmap='viridis')
plt.colorbar(im, ax=ax, label='Integrated Intensity')
ax.set_title(cube_info['name'] + ' - Moment 0')
plt.show()

# Save to FITS
# output_path = explorer_adv.save()
# print(f"Saved to: {output_path}")

## Performance Comparison: Standard vs Prefix-Sum

Compare computation times with and without prefix-sum optimization.

In [ ]:
import time

# Test parameters
test_params = {
    'moment': 'M0',
    'first': 30,
    'last': 70,
    'clip_sigma': 3.0,
    'use_mask': True
}

# Standard method
explorer_std = MomentMapExplorer()
explorer_std.load_cube(cube_path, mask_path)
explorer_std.enable_prefix_sums(False)

_, time_std = explorer_std.generate(**test_params)

# Prefix-sum method
explorer_prefix = MomentMapExplorer()
explorer_prefix.load_cube(cube_path, mask_path)
explorer_prefix.enable_prefix_sums(True)

_, time_prefix = explorer_prefix.generate(**test_params)

print(f"Standard method: {time_std*1000:.1f} ms")
print(f"Prefix-sum method: {time_prefix*1000:.1f} ms")
print(f"Speedup: {time_std/time_prefix:.2f}x")